In [1]:
import json
import os
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Optional
import math
import time

import numpy as np
import pandas as pd
import talib as ta
import mplfinance as mpf
from dotenv import load_dotenv
from okx.MarketData import MarketAPI
from pandas import DataFrame
from tqdm import tqdm

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

load_dotenv()
KEY = os.getenv("OKX_API_KEY")
SECRET = os.getenv("OKX_API_SECRET")
assert KEY and SECRET, "API key and secret are required"
market = MarketAPI(KEY, SECRET, flag="0", debug=False)

def to_candles(data: list) -> DataFrame:
    """将数据转换为用于绘制 K 线图的 DataFrame

    Args:
        data: JSON 数据

    Returns:
        DataFrame: 一组 K 线数据帧
    """
    df = pd.DataFrame(
        data,
        columns=[
            "ts",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "volCcy",
            "volCcyQuote",
            "_",
        ],
    )
    return df

def get_current_candlestick(instId: str, bar: str = "1H") -> DataFrame:
    """获取当前 K 线数据"""
    if not isinstance(instId, str):
        raise TypeError("instId must be a string")

    # 获取最近 3 个小时级别的 K 线数据
    resp = market.get_candlesticks(instId, bar=bar, limit="2")
    data = resp["data"]
    df = to_candles(data)
    return df


def get_period(bar):
    """
    根据给定的时间条形（bar）字符串，返回相应的时间周期（以毫秒为单位）。

    参数:
    bar (str): 时间条形字符串，例如 "1s", "1m", "3m", "5m", "15m", "1H", "2H", "4H", "1D"

    返回:
    int: 对应的时间周期，以毫秒为单位

    异常:
    ValueError: 如果输入的 bar 值不在预定义的范围内
    """
    if bar == "1s":
        period = 1 * 1000
    elif bar == "1m":
        period = 60 * 1000
    elif bar == "3m":
        period = 180 * 1000
    elif bar == "5m":
        period = 300 * 1000
    elif bar == "15m":
        period = 900 * 1000
    elif bar == "1H":
        period = 3600 * 1000
    elif bar == "2H":
        period = 7200 * 1000
    elif bar == "4H":
        period = 14400 * 1000
    elif bar == "1D":
        period = 86400 * 1000
    else:
        return 0
#         raise ValueError("bar must be one of 1s, 1m, 3m, 5m, 15m, 1H, 2H, 4H, 1D")

def get_candlesticks(instId: str, after: int | None = None, before: int | None = None, bar: str = "1H") -> pd.DataFrame:
    """获取 K 线数据, 包含历史数据和最新数据"""
    if not isinstance(instId, str):
        raise TypeError("instId must be a string")

    dst_dir = f"./data/{instId}/{bar}"
    file_path = f"./data/{instId}/{bar}/{instId}.json"
    period = get_period(bar)
    
    # 获取最新的 K 线数据
    latest_df = get_current_candlestick(instId, bar)
    history_data = []
    after_list = [int(after)]
    now_timestamp = datetime.now().timestamp() * 1000
    while 1:
        _after = after_list[-1] + period * 100
        after_list.append(_after)
        if _after > now_timestamp:
            break
            
    # 获取历史的 K 线数据
    for _after in tqdm(after_list):
        resp = market.get_history_candlesticks(instId, bar=bar, after=str(_after))
        data = resp["data"]
        history_data.extend(data)
        if len(data) == 0:
            break
        last_data = data[-1]
        time.sleep(0.1)
    
    history_df = to_candles(history_data)
    df = merge_candlesticks(history_df, latest_df)
    data = df.to_dict(orient="records")
    os.makedirs(dst_dir, exist_ok=True)
    with open(file_path, "w") as f:
        f.write(json.dumps(data, indent=4))
    return df

def merge_candlesticks(df1: DataFrame, df2: DataFrame) -> DataFrame:
    """合并两个 K 线数据帧

    Args:
        df1: K 线数据帧 1
        df2: K 线数据帧 2

    Returns:
        DataFrame: 合并后的 K 线数据帧
    """
    # 合并历史数据和最新数据，按时间戳去重，保留 '_' 为 1 的数据，即收盘数据
    df = pd.concat([df1, df2], ignore_index=True)
    df.drop_duplicates("ts", keep="last", inplace=True)
    df.sort_values("ts", ascending=True, inplace=True)
    return df

# 获取数据

In [50]:
instId = "SOL-USDT-SWAP"

# 获取过去一天的数据
start = datetime.now() - timedelta(days=1)
df = get_candlesticks(instId, bar="15m", after=start.timestamp() * 1000)
df.reset_index(drop=True, inplace=True)
df["ts"] = df["ts"].astype(int)
df["ts"] = pd.to_datetime(df["ts"], unit="ms") + timedelta(hours=8)
df["high"] = df["high"].astype(float)
df["low"] = df["low"].astype(float)
df["close"] = df["close"].astype(float)
df["hlc3"] = (df["high"] + df["low"] + df["close"]) / 3
df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.16it/s]


,ts,open,high,low,close,volume,volCcy,volCcyQuote,_,hlc3
0,2024-06-05 09:45:00,172.76,173.56,172.45,173.25,103925.8,103925.8,18002176.518,1,173.086667
1,2024-06-05 10:00:00,173.25,173.88,172.82,172.98,125270.2,125270.2,21727125.087,1,173.226667
2,2024-06-05 10:15:00,172.97,174.70,172.97,174.39,119753.3,119753.3,20832841.408,1,174.020000
3,2024-06-05 10:30:00,174.39,174.83,173.14,173.21,147803.8,147803.8,25736530.282,1,173.726667
4,2024-06-05 10:45:00,173.21,173.78,172.93,173.45,65417.6,65417.6,11341763.273,1,173.386667
...,...,...,...,...,...,...,...,...,...,...
191,2024-06-07 09:30:00,169.88,169.95,169.47,169.89,59335.21,59335.21,10065061.8633,1,169.770000
192,2024-06-07 09:45:00,169.89,170.38,169.86,170.20,23679.93,23679.93,4029845.4534,1,170.146667
193,2024-06-07 10:00:00,170.21,170.39,170.18,170.19,12212.18,12212.18,2079495.8144,1,170.253333
194,2024-06-07 10:15:00,170.18,170.19,169.67,169.87,29693.94,29693.94,5044116.1232,1,169.910000


# 策略

买卖点

In [51]:
rsiLen = 14
smoothLen = 2

# -----------------

inputMfiLen = 30
smoothHist = 2
fastMfiLen = round(inputMfiLen / 1.33)
slowMfiLen = round(inputMfiLen * 1.33)

# ---------------

MFI_LEN     = 7
STOCH_K     = 2
STOCH_D     = 5
SMOOTH_LEN  = 1.75
STOCH_WEIGHT = 0.4
OVERBOUGHT   = 60.0
EXTEND_MULT  = 1
mfiWeight    = 0.4


rsiLen = 14

# transform(src, mult=1)=>
#     tmp = (src / 100 - 0.5)*2
#     mult * 100 * ((tmp > 0 ? 1 : -1) * math.pow(math.abs(tmp), 0.75))

def transform(src, mult=1):
    tmp = (src / 100 - 0.5) * 2
    sign = np.where(tmp > 0, 1, -1)
    return mult * 100 * sign * np.power(np.abs(tmp), 0.75)

## Histogram

In [52]:
fastMfi = ta.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=fastMfiLen)
slowMfi = ta.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=slowMfiLen)
resMfi = transform(ta.SMA((fastMfi * 0.5 + slowMfi * 0.5), timeperiod=smoothHist), 0.7)

## Signal

In [53]:
mfi = ta.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=MFI_LEN)
rsi = ta.RSI(df['hlc3'], timeperiod=rsiLen)
# 计算Stochastic RSI
stoch_rsi_k, stoch_rsi_d = ta.STOCH(rsi, rsi, rsi, fastk_period=rsiLen, slowk_period=1, slowk_matype=0, slowd_period=1, slowd_matype=0)
# 计算STOCH_K的SMA
stoch = ta.SMA(stoch_rsi_k, timeperiod=STOCH_K)
sigStoch = ta.SMA(stoch, timeperiod=STOCH_D)
df["mfi"] = mfi
df["rsi"] = rsi
df["stoch"] = stoch
df["sigStoch"] = sigStoch
df[["ts", "mfi", "rsi", "stoch", "sigStoch"]][df["ts"] == "2024-06-06 20:00:00"]

,ts,mfi,rsi,stoch,sigStoch
137,2024-06-06 20:00:00,89.255382,58.349881,96.106056,70.103732


## 处理缺失的 volume 数据

In [54]:
# mfi[np.isnan(df['volume'])] = 0
# if np.isnan(df['volume']).any():
#     mfiWeight = 0

In [55]:
signal = (rsi + mfiWeight * mfi + STOCH_WEIGHT * stoch) / (1 + mfiWeight + STOCH_WEIGHT)
avg = transform(ta.EMA(signal, smoothLen), EXTEND_MULT)
avg2 = transform(ta.EMA(signal, round(smoothLen * SMOOTH_LEN)), EXTEND_MULT)

df['resMfi'] = resMfi
df['signal'] = signal
df['avg'] = avg
df['avg2'] = avg2
df[["ts", "resMfi", "signal", "avg", "avg2"]][df["ts"] == "2024-06-06 20:00:00"]

,ts,resMfi,signal,avg,avg2
137,2024-06-06 20:00:00,11.467776,73.608031,52.169552,42.560039


In [56]:
def pivot_high(series, left_bars, right_bars):
    """检查给定点是否是一个 pivot high，即它比左边和右边的 left_bars 和 right_bars 数量的点都高。"""
    pivots = [np.nan] * len(series)
    for i in range(left_bars, len(series) - right_bars):
        is_pivot = True
        for j in range(1, left_bars + 1):
            if series[i] <= series[i - j]:
                is_pivot = False
                break
        for j in range(1, right_bars + 1):
            if series[i] <= series[i + j]:
                is_pivot = False
                break
        if is_pivot:
            pivots[i+right_bars] = series[i]
    return pivots

def pivot_low(series, left_bars, right_bars):
    """检查给定点是否是一个 pivot low，即它比左边和右边的 left_bars 和 right_bars 数量的点都低。"""
    pivots = [np.nan] * len(series)
    for i in range(left_bars, len(series) - right_bars):
        is_pivot = True
        for j in range(1, left_bars + 1):
            if series[i] >= series[i - j]:
                is_pivot = False
                break
        for j in range(1, right_bars + 1):
            if series[i] >= series[i + j]:
                is_pivot = False
                break
        if is_pivot:
            pivots[i+right_bars] = series[i]
    return pivots
# @dataclass
# class Pivot:
#     price: float
#     ndx: int
#     indicator: float

@dataclass
class Point:
    price: float
    ndx: int

In [57]:
UP_BORDER = 50
DN_BORDER = -50
lastSigBar = 0

df["indPh"] = pivot_high(avg, 5, 5)
df["indPl"] = pivot_low(avg, 5, 5)
df[df["indPh"].notnull() | df["indPl"].notnull()][["ts", "indPh", "indPl", "open", "close"]]

,ts,indPh,indPl,open,close
34,2024-06-05 18:15:00,NaN,-39.108176,173.58,173.09
38,2024-06-05 19:15:00,7.775720,NaN,172.78,172.62
44,2024-06-05 20:45:00,NaN,-58.620457,173.02,173.01
51,2024-06-05 22:30:00,38.280720,NaN,172.46,172.82
55,2024-06-05 23:30:00,NaN,-46.301859,174.48,174.87
62,2024-06-06 01:15:00,69.265979,NaN,174.61,174.07
75,2024-06-06 04:30:00,NaN,-61.595777,173.22,173.10
88,2024-06-06 07:45:00,51.307665,NaN,173.53,173.57
92,2024-06-06 08:45:00,NaN,6.284340,173.62,173.70
106,2024-06-06 12:15:00,51.054787,NaN,173.36,173.09


In [58]:
# Initialize last pivot high and low
lastIndPh_price = np.nan
lastIndPh_ndx = np.nan
lastIndPl_price = np.nan
lastIndPl_ndx = np.nan

# Calculate speedH and speedL
speedH = np.full(avg.shape, np.nan)
speedL = np.full(avg.shape, np.nan)

for i in range(len(avg)):
    if not np.isnan(df["indPh"][i]):
        lastIndPh_price = df["indPh"][i]
        lastIndPh_ndx = i - 5
    if not np.isnan(df["indPl"][i]):
        lastIndPl_price = df["indPl"][i]
        lastIndPl_ndx = i - 5
    
    if not np.isnan(lastIndPh_price) and not np.isnan(lastIndPh_ndx):
        speedH[i] = (avg[i] - lastIndPh_price) / (i - lastIndPh_ndx)
    if not np.isnan(lastIndPl_price) and not np.isnan(lastIndPl_ndx):
        speedL[i] = (avg[i] - lastIndPl_price) / (i - lastIndPl_ndx)

# 将速度结果添加到 DataFrame
df['speedH'] = speedH
df['speedL'] = speedL
df[df["ts"] == "2024-06-06 18:00:00"][["ts", "speedH", "speedL"]]

,ts,speedH,speedL
129,2024-06-06 18:00:00,-1.170775,6.682367


In [59]:
sellSigRule = [False] * len(df)
buySigRule = [False] * len(df)
lastSigBar = 0

# 遍历 DataFrame 计算信号规则
for i in range(2, len(df)):
    if df['avg'][i] > UP_BORDER and df['avg'][i] > df['avg'][i-2] and df['speedH'][i] < df['speedH'][i-1] and (i - lastSigBar >= 10):
        sellSigRule[i] = True
        lastSigBar = i
    
    if df['avg'][i] < DN_BORDER and df['avg'][i] < df['avg'][i-2] and df['speedL'][i] > df['speedL'][i-1] and (i - lastSigBar >= 10):
        buySigRule[i] = True
        lastSigBar = i

# 将信号规则添加到 DataFrame
df['sellSigRule'] = sellSigRule
df['buySigRule'] = buySigRule
print("=== Buy Signal ===")
df[df["buySigRule"]][["ts", "buySigRule"]]
print("=== Sell Signal ===")
df[df["sellSigRule"]][["ts", "sellSigRule"]]

=== Buy Signal ===


,ts,buySigRule
107,2024-06-06 12:30:00,True
162,2024-06-07 02:15:00,True


=== Sell Signal ===


,ts,sellSigRule
58,2024-06-06 00:15:00,True
126,2024-06-06 17:15:00,True
